In [21]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [11]:
DATA_PATH = r"C:/Users/ZIRA/Desktop/MASTER/RP/mahir"
CITY = "New York"
WINDOW_SIZE = 24

In [12]:
csv_files = [
    "humidity.csv",
    "pressure.csv",
    "temperature.csv",
    "wind_direction.csv",
    "wind_speed.csv",
    "weather_description.csv"
]

def load_feature(file):
    df = pd.read_csv(os.path.join(DATA_PATH, file))
    df = df[["datetime", CITY]]
    df["datetime"] = pd.to_datetime(df["datetime"])
    feature_name = file.replace(".csv", "")
    return df.rename(columns={CITY: feature_name}).set_index("datetime")

#učitavanje svih csv-ova
dfs = [load_feature(f) for f in csv_files]

In [13]:
print(load_feature('humidity.csv'));

                     humidity
datetime                     
2012-10-01 12:00:00       NaN
2012-10-01 13:00:00      58.0
2012-10-01 14:00:00      57.0
2012-10-01 15:00:00      57.0
2012-10-01 16:00:00      57.0
...                       ...
2017-11-29 20:00:00       NaN
2017-11-29 21:00:00       NaN
2017-11-29 22:00:00       NaN
2017-11-29 23:00:00       NaN
2017-11-30 00:00:00       NaN

[45253 rows x 1 columns]


In [14]:
data = pd.concat(dfs, axis=1)

In [5]:
print(data)

                     humidity  pressure  temperature  wind_direction  \
datetime                                                               
2012-10-01 12:00:00       NaN       NaN          NaN             NaN   
2012-10-01 13:00:00      58.0    1012.0   288.220000           260.0   
2012-10-01 14:00:00      57.0    1012.0   288.247676           260.0   
2012-10-01 15:00:00      57.0    1012.0   288.326940           260.0   
2012-10-01 16:00:00      57.0    1012.0   288.406203           260.0   
...                       ...       ...          ...             ...   
2017-11-29 20:00:00       NaN       NaN          NaN             NaN   
2017-11-29 21:00:00       NaN       NaN          NaN             NaN   
2017-11-29 22:00:00       NaN       NaN          NaN             NaN   
2017-11-29 23:00:00       NaN       NaN          NaN             NaN   
2017-11-30 00:00:00       NaN       NaN          NaN             NaN   

                     wind_speed weather_description  
datetime 

In [16]:
data = data.ffill().bfill()

In [18]:
data = pd.get_dummies(data, columns=["weather_description"])

In [19]:
data = data.fillna(0)

In [22]:
data = data.astype(np.float32)

In [23]:
print(data)

                     humidity  pressure  temperature  wind_direction  \
datetime                                                               
2012-10-01 12:00:00      58.0    1012.0   288.220001           260.0   
2012-10-01 13:00:00      58.0    1012.0   288.220001           260.0   
2012-10-01 14:00:00      57.0    1012.0   288.247681           260.0   
2012-10-01 15:00:00      57.0    1012.0   288.326935           260.0   
2012-10-01 16:00:00      57.0    1012.0   288.406189           260.0   
...                       ...       ...          ...             ...   
2017-11-29 20:00:00      58.0    1020.0   284.980011             0.0   
2017-11-29 21:00:00      58.0    1020.0   284.980011             0.0   
2017-11-29 22:00:00      58.0    1020.0   284.980011             0.0   
2017-11-29 23:00:00      58.0    1020.0   284.980011             0.0   
2017-11-30 00:00:00      58.0    1020.0   284.980011             0.0   

                     wind_speed  weather_description_broken clo

In [27]:
numerical_cols = [col for col in data.columns if not col.startswith("weather_description_")]
scaler = MinMaxScaler()

In [25]:
print(numerical_cols)

['humidity', 'pressure', 'temperature', 'wind_direction', 'wind_speed']


In [28]:
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

In [29]:
print(data)

                     humidity  pressure  temperature  wind_direction  \
datetime                                                               
2012-10-01 12:00:00  0.533333  0.569893     0.629704        0.722222   
2012-10-01 13:00:00  0.533333  0.569893     0.629704        0.722222   
2012-10-01 14:00:00  0.522222  0.569893     0.630170        0.722222   
2012-10-01 15:00:00  0.522222  0.569893     0.631503        0.722222   
2012-10-01 16:00:00  0.522222  0.569893     0.632835        0.722222   
...                       ...       ...          ...             ...   
2017-11-29 20:00:00  0.533333  0.655914     0.575220        0.000000   
2017-11-29 21:00:00  0.533333  0.655914     0.575220        0.000000   
2017-11-29 22:00:00  0.533333  0.655914     0.575220        0.000000   
2017-11-29 23:00:00  0.533333  0.655914     0.575220        0.000000   
2017-11-30 00:00:00  0.533333  0.655914     0.575220        0.000000   

                     wind_speed  weather_description_broken clo

In [40]:
def create_sequences(data, target_col, window):
    X, y = [], []
    target_idx = data.columns.get_loc(target_col)
    print(data.columns)
    print(target_idx)
    print(len(data))
    print(len(data)-window)
    for i in range(len(data) - window):
        X.append(data.iloc[i:i+window].values)
        y.append(data.iloc[i+window, target_idx])
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    return X, y

In [41]:
target_column = "temperature"

X, y = create_sequences(data, target_column, WINDOW_SIZE)

Index(['humidity', 'pressure', 'temperature', 'wind_direction', 'wind_speed',
       'weather_description_broken clouds', 'weather_description_drizzle',
       'weather_description_dust', 'weather_description_few clouds',
       'weather_description_fog', 'weather_description_freezing rain',
       'weather_description_haze',
       'weather_description_heavy intensity drizzle',
       'weather_description_heavy intensity rain',
       'weather_description_heavy snow',
       'weather_description_heavy thunderstorm',
       'weather_description_light intensity drizzle',
       'weather_description_light intensity shower rain',
       'weather_description_light rain',
       'weather_description_light rain and snow',
       'weather_description_light snow', 'weather_description_mist',
       'weather_description_moderate rain',
       'weather_description_overcast clouds',
       'weather_description_proximity thunderstorm',
       'weather_description_proximity thunderstorm with drizzl

In [46]:
print(X[1][1])
print(y[1])

[0.5222222  0.5698929  0.63016987 0.72222227 0.28       0.
 0.         0.         1.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
0.66082716


In [33]:
print(y)

[0.6594944  0.66082716 0.6621599  ... 0.57521963 0.57521963 0.57521963]


In [51]:
print("X dtype:", X.dtype, "shape:", X.shape)
print("y dtype:", y.dtype, "shape:", y.shape)
print(len(data))

X dtype: float32 shape: (45229, 24, 41)
y dtype: float32 shape: (45229,)
45253


In [52]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [53]:
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(WINDOW_SIZE, X.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1) 
])

In [54]:

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.1)

pred = model.predict(X_test)

Epoch 1/20
509/509 [==============================] - 24s 42ms/step - loss: 0.0068 - val_loss: 0.0026
Epoch 2/20
509/509 [==============================] - 24s 47ms/step - loss: 0.0022 - val_loss: 0.0013
Epoch 3/20
509/509 [==============================] - 26s 52ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 4/20
509/509 [==============================] - 28s 56ms/step - loss: 0.0012 - val_loss: 6.4569e-04
Epoch 5/20
509/509 [==============================] - 29s 58ms/step - loss: 9.5186e-04 - val_loss: 4.4936e-04
Epoch 6/20
509/509 [==============================] - 30s 60ms/step - loss: 8.0674e-04 - val_loss: 6.0308e-04
Epoch 7/20
509/509 [==============================] - 27s 52ms/step - loss: 7.4317e-04 - val_loss: 3.2706e-04
Epoch 8/20
509/509 [==============================] - 29s 57ms/step - loss: 6.8779e-04 - val_loss: 3.5162e-04
Epoch 9/20
509/509 [==============================] - 28s 56ms/step - loss: 6.3818e-04 - val_loss: 4.8923e-04
Epoch 10/20
509/509 [=================

In [57]:
print(numerical_cols)

['humidity', 'pressure', 'temperature', 'wind_direction', 'wind_speed']


In [64]:
def invert_scale(pred_scaled, y_scaled, scaler, feature_name, numerical_cols):
    idx = numerical_cols.index(feature_name)
    print("PRED SCALED")
    print(pred_scaled)
    print("Y SCALED")
    print(y_scaled)
    pred_padded = np.zeros((len(pred_scaled), len(numerical_cols)))
    y_padded = np.zeros((len(y_scaled), len(numerical_cols)))
    print("PADDED PRED")
    print(pred_padded)
    print("PADDED Y")
    print(y_padded)
    pred_padded[:, idx] = pred_scaled.flatten()
    y_padded[:, idx] = y_scaled.flatten()
    print("PADDED PREDD")
    print(pred_padded)
    print("PADDED YY")
    print(y_padded)
    inv_pred = scaler.inverse_transform(pred_padded)[:, idx]
    inv_y = scaler.inverse_transform(y_padded)[:, idx]
    print(inv_pred)
    print(inv_y)
    return inv_pred, inv_y

In [65]:
pred_inv, y_test_inv = invert_scale(pred, y_test, scaler, target_column, numerical_cols)


PRED SCALED
[[0.51116735]
 [0.50064266]
 [0.4900846 ]
 ...
 [0.56905836]
 [0.56905836]
 [0.56905836]]
Y SCALED
[0.51770735 0.5062723  0.4857564  ... 0.57521963 0.57521963 0.57521963]
PADDED PRED
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
PADDED Y
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
PADDED PREDD
[[0.         0.         0.51116735 0.         0.        ]
 [0.         0.         0.50064266 0.         0.        ]
 [0.         0.         0.49008459 0.         0.        ]
 ...
 [0.         0.         0.56905836 0.         0.        ]
 [0.         0.         0.56905836 0.         0.        ]
 [0.         0.         0.56905836 0.         0.        ]]
PADDED YY
[[0.         0.         0.51770735 0.         0.        ]
 [0.         0.         0.50627232 0.         0.        ]
 [0.         0.         0.4857564  0.         0.        ]
 ...
 [0.        